# 线性回归模型调试任务

## 欢迎来到AI世界！

在这个考试任务中，我们将一起调试一个线性回归模型。我们将使用加利福尼亚房价数据集，该数据集包含了如收入中位数、房龄、房间数量、地理位置等8个特征，我们的目标是构建一个模型来准确预测区域的房价中位数。线性回归是机器学习中最基础的算法之一，就像学习数学要先学会加减法一样重要！

### 我们的目标：
- 理解线性回归的基本原理
- 学会使用梯度下降算法
- 掌握PyTorch的基本操作
- 通过调试找出代码中的问题并修复

## <font color='red'><b>🔍 调试任务说明</b></font>

<font color='red'><b>任务目标：这个项目中有几处代码需要调试和修复。你的任务是：</b></font>
<br>
<font color='red'><b>1. 逐步执行此文档里的每一步代码</b></font>
<br>
<font color='red'><b>2. 理解代码的含义</b></font>
<br>
<font color='red'><b>3. 根据第6步和第7步代码的提示，修改相应的参数，使其达到任务设定值</b></font>

评分规则说明：验证准确率在
<br>
只完成任务1，未完成任务2，给10分；
<br>
只完成任务2，未完成任务1，给15分；
<br>
完成任务1和任务2，给25分。

## 第一步：导入必要的工具包

In [54]:
import numpy as np  # 数学计算工具箱
import pandas as pd  # 数据处理工具箱
import torch  # 深度学习框架
import torch.nn as nn  # 神经网络模块
import matplotlib.pyplot as plt  # 绘图工具箱
%matplotlib inline

import matplotlib

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置图像大小和分辨率
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

print("所有工具包导入成功！")
print(f"PyTorch版本: {torch.__version__}")

所有工具包导入成功！
PyTorch版本: 2.8.0+cpu


## 第二步：加载和了解数据

我们将使用波士顿房价数据集，这是一个经典的回归问题数据集。

In [55]:
# 使用加利福尼亚房价数据集作为替代
from sklearn.datasets import fetch_california_housing
import pandas as pd
from sklearn.model_selection import train_test_split

# 加载加利福尼亚房价数据集
california = fetch_california_housing()
X, y = california.data, california.target

# 查看数据集信息
print("特征名称:", california.feature_names)
print("数据形状:", X.shape)
print("目标变量形状:", y.shape)

# 转换为DataFrame便于查看
df = pd.DataFrame(X, columns=california.feature_names)
df['Price'] = y
print(df.head())

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 查看数据的基本信息
print("=== 数据基本信息 ===")
print(f"训练集特征形状: {X_train.shape}")
print(f"训练集标签形状: {y_train.shape}")
print(f"测试集特征形状: {X_test.shape}")
print(f"测试集标签形状: {y_test.shape}")
print(f"特征数量: {X_train.shape[1]}")
print(f"特征名称: {california.feature_names}")  # 移除了 .tolist()

# 显示前5个样本
print("\n=== 前5个样本的特征值 ===")
for i in range(5):
    print(f"样本 {i+1}: {X_train[i]} -> 价格: {y_train[i]:.2f}千美元")

# 数据统计信息
print("\n=== 数据统计信息 ===")
print(f"房价范围: {y.min():.2f} ~ {y.max():.2f}千美元")
print(f"平均房价: {y.mean():.2f}千美元")

特征名称: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
数据形状: (20640, 8)
目标变量形状: (20640,)
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  Price  
0    -122.23  4.526  
1    -122.22  3.585  
2    -122.24  3.521  
3    -122.25  3.413  
4    -122.25  3.422  
=== 数据基本信息 ===
训练集特征形状: (16512, 8)
训练集标签形状: (16512,)
测试集特征形状: (4128, 8)
测试集标签形状: (4128,)
特征数量: 8
特征名称: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']

=== 前5个样本的特征值 ===
样本 1: [ 3.25960000e+00  3.30000000e+01  5.01765650e

## 第三步：数据预处理

为了让模型更好地学习，我们需要对数据进行标准化处理。

In [56]:
# 数据标准化 - 这是一个重要的步骤！
from sklearn.preprocessing import StandardScaler

# 创建标准化器
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# 对训练集进行标准化
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()

# 对测试集进行标准化（使用训练集的参数）
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

# 转换为PyTorch张量
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test_scaled)

print("=== 数据预处理完成 ===")
print(f"标准化后的训练集特征形状: {X_train_tensor.shape}")
print(f"标准化后的训练集标签形状: {y_train_tensor.shape}")
print(f"标准化后的特征均值: {X_train_tensor.mean():.4f}")
print(f"标准化后的特征标准差: {X_train_tensor.std():.4f}")

=== 数据预处理完成 ===
标准化后的训练集特征形状: torch.Size([16512, 8])
标准化后的训练集标签形状: torch.Size([16512])
标准化后的特征均值: 0.0000
标准化后的特征标准差: 1.0000


## 第四步：理解线性回归原理

### 线性回归公式：
$$ y = w_1x_1 + w_2x_2 + ... + w_nx_n + b $$

其中：
- $x_1, x_2, ..., x_n$ 是特征（输入）
- $w_1, w_2, ..., w_n$ 是权重（需要学习的参数）
- $b$ 是偏置项
- $y$ 是预测值（输出）

In [57]:
# 函数1：手动实现线性回归预测函数
def linear_regression_predict(X, weights, bias):
    """
    手动实现线性回归预测
    
    参数:
    X: 输入特征，形状为 (样本数, 特征数)
    weights: 权重，形状为 (特征数,)
    bias: 偏置，标量
    
    返回:
    y_pred: 预测值，形状为 (样本数,)
    """
    # TODO: 实现线性回归的预测公式
    # 提示：使用 torch.matmul 或 @ 运算符进行矩阵乘法
    # 注意：这里有一个需要调试的问题
    y_pred = torch.matmul(X, weights) + bias  # 请检查这行代码是否有问题
    return y_pred

# 测试预测函数
sample_weights = torch.randn(X_train_tensor.shape[1])  # 随机初始化权重
sample_bias = torch.randn(1)  # 随机初始化偏置

predictions = linear_regression_predict(X_train_tensor, sample_weights, sample_bias)
print(f"预测值形状: {predictions.shape}")
print(f"前5个预测值: {predictions[:5]}")
print(f"对应的真实值: {y_train_tensor[:5]}")

预测值形状: torch.Size([16512])
前5个预测值: tensor([-0.8671, -3.0281,  2.9226, -2.4552, -2.3297])
对应的真实值: tensor([-0.9012,  1.5128, -0.2992, -0.9842, -0.9574])


## 第五步：理解损失函数

### 均方误差（MSE）公式：
$$ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$

其中：
- $y_i$ 是真实值
- $\hat{y}_i$ 是预测值
- $n$ 是样本数量

In [58]:
# 函数2：手动实现均方误差损失函数
def mse_loss(y_true, y_pred):
    """
    计算均方误差损失
    
    参数:
    y_true: 真实值，形状为 (样本数,)
    y_pred: 预测值，形状为 (样本数,)
    
    返回:
    loss: 均方误差损失，标量
    """
    # TODO: 实现均方误差公式
    # 提示：先计算差值，然后平方，最后求平均
    # 注意：这里有一个需要调试的问题
    loss = torch.mean((y_true - y_pred) ** 2)  # 请检查这行代码是否有问题
    return loss

# 测试损失函数
test_loss = mse_loss(y_train_tensor, predictions)
print(f"当前模型的损失: {test_loss:.4f}")

# 使用PyTorch内置函数验证
torch_loss = nn.MSELoss()(predictions, y_train_tensor)
print(f"PyTorch内置损失函数结果: {torch_loss:.4f}")
print(f"我们的实现是否正确: {torch.abs(test_loss - torch_loss) < 1e-6}")

当前模型的损失: 6.6775
PyTorch内置损失函数结果: 6.6775
我们的实现是否正确: True


## 第六步：采用梯度下降对线性回归模型进行参数训练

梯度下降是通过计算损失函数对参数的梯度（导数），然后沿着梯度反方向更新参数来最小化损失函数的过程。

### 参数更新公式：
$$ w = w - \alpha \cdot \frac{\partial L}{\partial w} $$
$$ b = b - \alpha \cdot \frac{\partial L}{\partial b} $$

其中：
- $\alpha$ 是学习率
- $\frac{\partial L}{\partial w}$ 是损失对权重的梯度
- $\frac{\partial L}{\partial b}$ 是损失对偏置的梯度

<div style="color: #FF0000; font-size: 18px; font-weight: bold; background-color: #FFF0F0; padding: 10px; border: 2px solid #FF0000; border-radius: 5px;"> 考试任务1：下面的函数3中，其它参数不变，修改学习率，使得最后的损失函数值在0.4以下 </div>

In [64]:
# 函数3：手动实现梯度下降
def manual_gradient_descent(X, y, weights, bias, learning_rate, epochs):
    """
    手动实现梯度下降
    
    参数:
    X: 输入特征
    y: 真实标签
    weights: 初始权重
    bias: 初始偏置
    learning_rate: 学习率
    epochs: 训练轮数
    
    返回:
    weights: 训练后的权重
    bias: 训练后的偏置
    losses: 每轮的损失记录
    """
    losses = []
    
    for epoch in range(epochs):
        # 前向传播：计算预测值
        y_pred = linear_regression_predict(X, weights, bias)
        
        # 计算损失
        loss = mse_loss(y, y_pred)
        losses.append(loss.item())
        
        # 手动计算梯度
        # 对于线性回归，梯度的计算公式为：
        # dL/dw = (2/n) * X^T @ (y_pred - y)
        # dL/db = (2/n) * sum(y_pred - y)
        
        n = len(y)
        error = y_pred - y
        
        # TODO: 计算权重和偏置的梯度
        # 注意：这里有两个需要调试的问题
        grad_weights = (2/n) * torch.matmul(X.T, error)  # 请检查这行代码
        grad_bias = (2/n) * torch.sum(error)  # 请检查这行代码
        
        # 更新参数
        weights = weights - learning_rate * grad_weights
        bias = bias - learning_rate * grad_bias
        
        if epoch % 2 == 0:
            print(f"轮次 {epoch}, 损失: {loss:.4f}")
    
    return weights, bias, losses

# 测试手动梯度下降
print("=== 开始手动梯度下降训练 ===")
manual_weights = torch.randn(X_train_tensor.shape[1], requires_grad=False)
manual_bias = torch.randn(1, requires_grad=False)

final_weights, final_bias, manual_losses = manual_gradient_descent(
    X_train_tensor, y_train_tensor, manual_weights, manual_bias,
    # todo: 修改学习率到0.3，然后多跑几次找到最好成绩
    learning_rate=0.3, epochs=10
)

print(f"\n训练完成！最终损失: {manual_losses[-1]:.4f}")

=== 开始手动梯度下降训练 ===
轮次 0, 损失: 2.4012
轮次 2, 损失: 0.4410
轮次 4, 损失: 0.4024
轮次 6, 损失: 0.3968
轮次 8, 损失: 0.3948

训练完成！最终损失: 0.3941


<div style="color: #FF0000; font-size: 18px; font-weight: bold; background-color: #FFF0F0; padding: 10px; border: 2px solid #FF0000; border-radius: 5px;"> 考试任务2：下面的函数4中，其它参数不变，修改训练轮数，使得最后的损失函数值在0.4以下 </div>

In [67]:
# 函数4：手动实现梯度下降
def manual_gradient_descent(X, y, weights, bias, learning_rate, epochs):
    """
    手动实现梯度下降
    
    参数:
    X: 输入特征
    y: 真实标签
    weights: 初始权重
    bias: 初始偏置
    learning_rate: 学习率
    epochs: 训练轮数
    
    返回:
    weights: 训练后的权重
    bias: 训练后的偏置
    losses: 每轮的损失记录
    """
    losses = []
    
    for epoch in range(epochs):
        # 前向传播：计算预测值
        y_pred = linear_regression_predict(X, weights, bias)
        
        # 计算损失
        loss = mse_loss(y, y_pred)
        losses.append(loss.item())
        
        # 手动计算梯度
        # 对于线性回归，梯度的计算公式为：
        # dL/dw = (2/n) * X^T @ (y_pred - y)
        # dL/db = (2/n) * sum(y_pred - y)
        
        n = len(y)
        error = y_pred - y
        
        # TODO: 计算权重和偏置的梯度
        # 注意：这里有两个需要调试的问题
        grad_weights = (2/n) * torch.matmul(X.T, error)  # 请检查这行代码
        grad_bias = (2/n) * torch.sum(error)  # 请检查这行代码
        
        # 更新参数
        weights = weights - learning_rate * grad_weights
        bias = bias - learning_rate * grad_bias
        
        if epoch % 2 == 0:
            print(f"轮次 {epoch}, 损失: {loss:.4f}")
    
    return weights, bias, losses

# 测试手动梯度下降
print("=== 开始手动梯度下降训练 ===")
manual_weights = torch.randn(X_train_tensor.shape[1], requires_grad=False)
manual_bias = torch.randn(1, requires_grad=False)

final_weights, final_bias, manual_losses = manual_gradient_descent(
    X_train_tensor, y_train_tensor, manual_weights, manual_bias,
    # todo: 修改训练轮数到1000, 大力出奇迹
    learning_rate=0.01, epochs=1000
)

print(f"\n训练完成！最终损失: {manual_losses[-1]:.4f}")

=== 开始手动梯度下降训练 ===
轮次 0, 损失: 15.4763
轮次 2, 损失: 13.9491
轮次 4, 损失: 12.5886
轮次 6, 损失: 11.3753
轮次 8, 损失: 10.2923
轮次 10, 损失: 9.3248
轮次 12, 损失: 8.4597
轮次 14, 损失: 7.6853
轮次 16, 损失: 6.9917
轮次 18, 损失: 6.3698
轮次 20, 损失: 5.8117
轮次 22, 损失: 5.3106
轮次 24, 损失: 4.8601
轮次 26, 损失: 4.4549
轮次 28, 损失: 4.0901
轮次 30, 损失: 3.7613
轮次 32, 损失: 3.4649
轮次 34, 损失: 3.1974
轮次 36, 损失: 2.9558
轮次 38, 损失: 2.7375
轮次 40, 损失: 2.5400
轮次 42, 损失: 2.3612
轮次 44, 损失: 2.1992
轮次 46, 损失: 2.0524
轮次 48, 损失: 1.9192
轮次 50, 损失: 1.7982
轮次 52, 损失: 1.6883
轮次 54, 损失: 1.5883
轮次 56, 损失: 1.4974
轮次 58, 损失: 1.4146
轮次 60, 损失: 1.3391
轮次 62, 损失: 1.2703
轮次 64, 损失: 1.2074
轮次 66, 损失: 1.1500
轮次 68, 损失: 1.0976
轮次 70, 损失: 1.0496
轮次 72, 损失: 1.0056
轮次 74, 损失: 0.9654
轮次 76, 损失: 0.9285
轮次 78, 损失: 0.8946
轮次 80, 损失: 0.8635
轮次 82, 损失: 0.8349
轮次 84, 损失: 0.8086
轮次 86, 损失: 0.7844
轮次 88, 损失: 0.7621
轮次 90, 损失: 0.7415
轮次 92, 损失: 0.7225
轮次 94, 损失: 0.7050
轮次 96, 损失: 0.6888
轮次 98, 损失: 0.6738
轮次 100, 损失: 0.6599
轮次 102, 损失: 0.6470
轮次 104, 损失: 0.6351
轮次 106, 损失: 0.6240
轮次 10